# Using iceflow with icepyx to Generate an Elevation Timeseries

This notebook shows how to produce a harmonized elevation timeseries across all iceflow-supported datasets along with ICESat-2 data using [icepyx](https://github.com/icesat2py/icepyx).

If this is your first time using `iceflow`, we recommend starting with the [NSIDC Iceflow example Jupyter Notebook](https://iceflow.readthedocs.io/en/latest/iceflow-example.html) first.

Similarly, if you are new to `icepyx`, we suggest reviewing the [icepyx documentation](https://icepyx.readthedocs.io/en/latest/) for more information about how to use `icepyx`.

## Scenario: assessing ice surface elevation change near Sermeq Kujalleq (Jakobshavn Isbrae)

In this notebook, we will focus on a small area near Sermeq Kujalleq (Jakobshavn Isbrae).

This notebook will demonstrate how to:

* Search for and download all iceflow-supported data for our area of interest and timeframe.
* Search for and download ICESat-2 data using `icepyx`.
* Average elevation data over our area of interest at a weekly resolution.
* Plot the results as a timeseries.

### Import required packages

We will import several packages needed to perform our analysis:

In [ ]:
# Imports
from pathlib import Path
import datetime as dt

import dask.dataframe as dd
import icepyx as ipx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

from nsidc.iceflow import (
    make_iceflow_parquet,
    DatasetSearchParameters,
    BoundingBox,
    find_iceflow_data,
    download_iceflow_results,
    IceflowDataFrame
)

### Search for and download iceflow-supported data for our area of interest and timeframe.

We first need to define some constants, including our download path, ITRF, and filter parameters. These will be used throughout the rest of the notebook.

**Note that the data supported by `iceflow` and `icepyx` can be very large! This tutorial will download ~2GB worth of data to local disk.**

In [ ]:
# Constants

# All of our data will be downloaded to this location. 
OUTPUT_DIR =  Path("./downloaded-data/")

# ICESat2 data products use ITRF2014 (e.g., see https://nsidc.org/data/atl06/versions/6):
# > WGS 84 ellipsoid, ITRF2014 reference frame
# NOTE/TODO: This is expected to change in the near future! ICESat2 release
# 7, scheduled for spring 2025, is expected to be referenced to ITRF2020.
ICESAT2_ITRF = "ITRF2014"

# This bounding box covers an area near Sermeq Kujalleq (Jakobshavn Isbrae)
BBOX = BoundingBox(lower_left_lon=-49.149, lower_left_lat=69.186, upper_right_lon=-48.949, upper_right_lat=69.238)

# Range of dates we want to evaluate
DATE_RANGE = (dt.date(2007, 1, 1), dt.date(2024, 10, 28))

Next we will use the `find_iceflow_data` function from the `iceflow` API to find data matching our area of interest.

By default, `DatasetSearchParameters` will include all `iceflow` supported datasets, unless one or more are specified as a filter with the `datasets` kwarg. There may be warnings raised about there not being search results for specific datasets supported by `iceflow`.

In [ ]:
search_results = find_iceflow_data(
    dataset_search_params=DatasetSearchParameters(
        bounding_box=BBOX,
        temporal=DATE_RANGE,
    ),
)
len(search_results)

Now download the matching files to disk using `download_iceflow_results`.

**Note: This next step may take a while, and will download data to your local disk.**

In [ ]:
downloaded_files = download_iceflow_results(search_results, output_dir=OUTPUT_DIR)

Finally, we will create a parquet dataset using `make_iceflow_parquet`.Writing data to a parquet dataset allows `dask` (which we will use later!) to read the chunks of data it needs to do calculations (e.g., `mean`) without needing to read all of the data into memory at once. This is important because `iceflow` can find many millions of data points for even small areas of interest!

This function transforms all data found in the `data_dir` to the provided `target_itrf`, further facilitating analysis across many datasets. 

**Note: This next step may take a while**

In [ ]:
parquet_path = make_iceflow_parquet(
    data_dir=OUTPUT_DIR,
    target_itrf=ICESAT2_ITRF,
    overwrite=True,
)

Now we read the data stored in the parquet dataset using dask:

In [ ]:
iceflow_df = dd.read_parquet(parquet_path)

# Ensure that our index is set as a datetime object 
iceflow_df = iceflow_df.reset_index()
iceflow_df["utc_datetime"] = dd.to_datetime(iceflow_df["utc_datetime"])
iceflow_df = iceflow_df.set_index("utc_datetime")

iceflow_df.head()

###  Search for and download ICESat-2 data using `icepyx`

Next, we will use icepyx to find [ATL06](https://nsidc.org/data/atl06/versions/6) data for the same area of interest and timeframe. 

To learn more about `icepyx.Query`, which is used below, see the [documentation](https://icepyx.readthedocs.io/en/latest/_icepyx/icepyx.Query.html#icepyx.Query).

In [ ]:
# We will compare the ILATM1B data to ATL06 data from October 2018.
result = ipx.Query(
    "ATL06",
    [
        BBOX.lower_left_lon,
        BBOX.lower_left_lat,
        BBOX.upper_right_lon,
        BBOX.upper_right_lat,
    ],
    DATE_RANGE,
)
result

Now we download the results.  Note that this may take a while to download data to local disk.

**Note: This next step may take a while**

In [ ]:
result.download_granules("downloaded-data/ATL06/")

Download complete


###  Read ICESat-2 data into `xarray` and `pandas`

Next, we will use [icepyx.Read](https://icepyx.readthedocs.io/en/latest/_icepyx/icepyx.Read.html#icepyx.Read) to read the data into an [xarray Dataset](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html).

Note that the code below wraps the reading of each file in a `try/except` block because of a known issue with subsetting. See https://github.com/icesat2py/icepyx/issues/576 for more information.

In [ ]:
datasets = []
for file in Path("downloaded-data/ATL06/").glob("*.h5"):
    try:
        reader = ipx.Read(str(file))
        reader.vars.append(var_list=["h_li", "latitude", "longitude"])
        ds = reader.load()
        datasets.append(ds)
    except:
        print(f"{file=} contains an error and will not be read")
        continue

len(datasets)

In [ ]:
ds = xr.concat(datasets, dim="gran_idx")
ds.head()

`icepyx` reads ICESat-2 data as an xarray dataset. `xarray` is a powerful tool and the data is ready to use in this format, but to simplify things for this notebook and make the data more compatible with `iceflow`, the next step will convert the data into an `iceflow`-compatible [pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html).

The first step is to rename the variables to be consistent with how `iceflow` identifies the elevation and time fields:

In [ ]:
ds = ds.rename({"h_li": "elevation", "delta_time": "utc_datetime"})

The following code block converts the `xarray` dataset into an iceflow dataframe:

In [ ]:
spot_dfs = []
for spot in ds.spot.data.flatten():
    df = pd.DataFrame.from_dict(
        {
            "elevation": ds.sel(spot=spot).elevation.data.flatten(),
            "latitude": ds.sel(spot=spot).latitude.data.flatten(),
            "longitude": ds.sel(spot=spot).longitude.data.flatten(),
            "utc_datetime": ds.sel(spot=spot).utc_datetime.data.flatten(),
            "spot": [spot] * len(ds.sel(spot=spot).elevation.data.flatten()),
            "ITRF": ICESAT2_ITRF,
        },
    )
    spot_dfs.append(df)


df = pd.concat(spot_dfs, sort=True)
# Drop rows where lat, lon, or elev are missing.
df = df.dropna(subset=["latitude", "longitude", "elevation"], how="any")
df = df.set_index("utc_datetime")
# Cast the df as an IceflowDataFrame. 
# The `atl06_df` can now be used with e.g., `iceflow`'s ITRF conversion function 
# to perform plate motion model adjustments if desired.
atl06_df = IceflowDataFrame(df)
atl06_df.head()

The ATL06 data contains some negative elevation values, as we see printed below. We will filter these out, as we expect positive elevations.

In [ ]:
print(atl06_df.elevation.min())

# Filter out negative values. We expect positive elevations.
atl06_df = atl06_df[atl06_df.elevation > 0]

### Average elevation data over our area of interest at a weekly resolution

In this next step, we will resample the ATL06 and `iceflow` data to a weekly resolution, taking the mean of elevation over our area of interest. This will provide us with one data point per week where data is available, giving us a general idea of how the elevation of our area of interest changes over time. First we resample ATL06 data to a weekly mean:

In [ ]:
atl06_avg_df = atl06_df[["elevation"]].resample("W").mean()
atl06_avg_df = atl06_avg_df.dropna(how="any")
atl06_avg_df.head()

And now we resample the iceflow data to a weekly mean:

In [ ]:
iceflow_df_sampled = iceflow_df.repartition(freq="1W")
iceflow_df_sampled = iceflow_df_sampled.dropna(how="any")

iceflow_df_sampled = iceflow_df_sampled[iceflow_df_sampled.elevation > 0]

iceflow_avg = iceflow_df_sampled.resample("W").agg({
    "elevation": "mean",
    "dataset": lambda x: ", ".join(x.astype("str").unique()),   
})
iceflow_avg = iceflow_avg.replace([np.inf, -np.inf], np.nan) 
iceflow_avg = iceflow_avg.dropna(how="any")
iceflow_avg = iceflow_avg.compute()

iceflow_avg.head()

### Plot the results as a timeseries

Now we will use [matplotlib](https://matplotlib.org/) to plot the results as a timeseries:

In [ ]:
%matplotlib inline

# We will use a unique marker for each dataset
iceflow_marker_map = {
    "GLAH06v034": "s",
    "BLATM1Bv1": "D",
    "ILATM1Bv1": "x",
    "ILATM1Bv2": "o",
    "ILVIS2v1": "v",
    "ILVIS2v2": "^",
}

fig = plt.figure(figsize=(10, 8))

for dataset, marker in iceflow_marker_map.items():
    subset = iceflow_avg[iceflow_avg.dataset == dataset]
    if subset.elevation.any():
        plt.scatter(subset.index, subset.elevation, marker=marker, label=dataset, linestyle="", color="black")

plt.scatter(atl06_avg_df.index, atl06_avg_df.elevation, color="black", marker="*", label="ATL06v6", linestyle="")

plt.xlabel("Date")
plt.ylabel("Elevation")
plt.legend(title="Dataset")

## Conclusions

In this notebook, we found and analyzed laser altimetry data from a variety of datasets using `iceflow` and `icepyx`. 

In the timeseries plot above, we can see how the surface elevation of a small area near Sermeq Kujalleq (Jakobshavn Isbrae) changes over time.

Note that this analysis was relatively simple. Although the data and plot above give us an idea of surface elevation changes, it should be noted that there are still a number of things a researcher should consider when doing an analysis across many datasets over time. To further this analysis, we may want to consider doing one or more of the following:

* Outlier detection and filtering
* Cross-calibration of data between sensors/datasets
* Plate motion model coordinate propagation (see the [NSIDC Iceflow example Jupyter Notebook](https://iceflow.readthedocs.io/en/latest/iceflow-example.html) for an example of how to do this)
* Account for spatial variability within our region of interest